In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import Settings, SummaryIndex, load_index_from_storage, StorageContext, Settings, \
    SimpleDirectoryReader, VectorStoreIndex, DocumentSummaryIndex, get_response_synthesizer
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
import time
import chromadb

from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.retrievers import SummaryIndexLLMRetriever
from llama_index.retrievers.bm25 import BM25Retriever

from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.tools import RetrieverTool
from llama_index.core.response.notebook_utils import display_source_node


In [7]:
ollama_embedding = OllamaEmbedding(
    model_name="llama2",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
    embed_batch_size=100
)

llm = Settings.llm = Ollama(base_url="http://localhost:11434",model="llama2",)
Settings.embed_model = ollama_embedding
t1=time.time()
documents = SimpleDirectoryReader("G:\data\Chinese-medical-dialogue-data-master\Data_statistic\sample").load_data()
index = VectorStoreIndex.from_documents(documents)
t2=time.time()

print("index_time", t2-t1)


index_time 68.5378475189209


In [12]:
query_str = "What would the author say about art vs. engineering?"
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
t3=time.time()
display(Markdown(f"<b>{response}</b>"))

print("query_time1", t3-t2,"-----------------------------------")

hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

print("query_time2", t3-t2,"-----------------------------------")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b>The author does not directly mention art versus engineering in the provided text. However, based on the context, it is possible to make some inferences.

The author seems to have a background in computer science and programming, as they mention working on Y Combinator and writing essays about startups. They also mention wanting to paint after their mother's death, which suggests an interest in creative pursuits.

On the other hand, the author also mentions being focused on their work and feeling a sense of responsibility to their mother during her illness. This suggests that they may prioritize practical and responsible pursuits over more creative or artistic ones.

Overall, it is difficult to say definitively what the author would say about art versus engineering based solely on the provided text. However, it seems likely that they would view engineering as a more practical and responsible field, while reserving a more creative or artistic outlet for their personal interests.</b>

query_time1 606.1916863918304 -----------------------------------
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


<b>Based on the context of the article, the author's views on art versus engineering are not directly discussed. However, some themes related to creativity, problem-solving, and the intersection of technology and humanities can be gleaned from the text. Here are some possible insights into the author's perspective:

1. The author seems to value creativity and problem-solving skills equally with technical expertise. In the passage about Y Combinator, the author highlights Sam Altman's ability to start a startup to make nuclear reactors, indicating that he views entrepreneurship as a creative pursuit alongside engineering.
2. The author recognizes the importance of attention and focus in any creative pursuit. When discussing his painting hobby, he notes that he was rusty at first but eventually got back into shape, suggesting that he values the process of dedicating oneself to a creative activity.
3. The author seems fascinated by the intersection of technology and humanities, as evidenced by his interest in Lisp and its origins as a model of computation. This curiosity about the foundations of programming languages suggests an appreciation for the interplay between technical and intellectual pursuits.
4. The author's personal journey also touches on the idea that there can be value in exploring different creative outlets, even if they seem unrelated to one's main field of expertise. His decision to take up painting after his mother's death and his subsequent engagement with the activity suggest that he finds joy and fulfillment in expressing himself through different forms of creativity.

In summary, while the author does not explicitly address art versus engineering, his views on creativity, problem-solving, and the interplay between technology and humanities suggest a nuanced appreciation for the role of both technical and intellectual pursuits in his life.</b>

query_time2 606.1916863918304 -----------------------------------


In [17]:
import pandas as pd

from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response

data_generator = DatasetGenerator.from_documents(documents)

eval_questions = data_generator.generate_questions_from_nodes()

eval_questions

E:\anaconda\envs\llamaindex_llama\Lib\site-packages\llama_index\core\evaluation\dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


RuntimeError: asyncio.run() cannot be called from a running event loop

In [16]:
E:\anaconda\envs\llamaindex_llama\python.exe -m pip install spacy

SyntaxError: unexpected character after line continuation character (579703732.py, line 1)